Chuẩn bị thư viện

In [5]:
!pip3 install tensorflow_text==1.15

     |████████████████████████████████| 9.1MB 2.7MB/s 
     |████████████████████████████████| 110.5MB 91kB/s 
     |████████████████████████████████| 3.8MB 44.7MB/s 
     |████████████████████████████████| 512kB 40.4MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7e8781accb012383c611cb5d6213caeaa9be09a639d4ce0656e89f7305225e08
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.2
    Uninstalling tensorboard-2.2.2:
      Successfully uninstalled tensorboard-2.2.2
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorflow-estimator 2.2.0
    Uninstalling tensorflow-estimator-2.2.0:
      Successfully uninstalled ten

In [6]:
!pip install tensorflow==1.15


     |████████████████████████████████| 412.3MB 40kB/s 
  Found existing installation: tensorflow 1.15.3
    Uninstalling tensorflow-1.15.3:
      Successfully uninstalled tensorflow-1.15.3


In [1]:
import string
import nltk
nltk.download('wordnet')
nltk.download('brown')
from nltk.corpus import brown
from gensim.models import Word2Vec
import multiprocessing
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import tensorflow_hub as hub
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Lambda, Dense, LSTM, Embedding
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Model

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# path
checkpoint_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/output/{0}'
data_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/data/{0}'
model_path = '/content/drive/My Drive/Colab Notebooks/Language Model/Language Models/embedding model/{0}'
url = "https://tfhub.dev/google/elmo/2"

# **Phần 0: Dữ liệu**

# **Phần 1: Elmo Modeling**

# apply elmo

In [4]:
embed = hub.Module(url)

In [5]:
# just a random sentence
x = ["Roasted ants are a popular snack in Columbia"]

# Extract ELMo features 
embeddings =embed(x, signature="default", as_dict=True)["elmo"]

embeddings.shape

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


TensorShape([Dimension(1), Dimension(8), Dimension(1024)])

In [6]:

def elmo_vectors(x):
  embeddings = elmo(x.tolist(), signature="default", as_dict=True)["elmo"]

  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    # return average of ELMo features
    return sess.run(tf.reduce_mean(embeddings,1))

In [7]:
# elmo_train = [elmo_vectors(x) for x in doc]
# elmo_test = [elmo_vectors(x['clean_tweet']) for x in list_test]


# **Phần 2: Word-based Modeling**

# Load lại tập chuỗi

In [8]:
# load doc into memory
def load_doc(filename):
	# open the file as read only
	file = open(filename, 'r')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load
in_filename = data_path.format('republic_sequences.txt')
doc = load_doc(in_filename)
lines = doc.split('\n')
print(lines[:2])

['the project gutenberg ebook of the republic by plato this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online', 'project gutenberg ebook of the republic by plato this ebook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever you may copy it give it away or reuse it under the terms of the project gutenberg license included with this ebook or online at']


# Encode tập chuỗi

In [9]:
# integer encode sequences of words
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)
sequences = tokenizer.texts_to_sequences(lines)
print(sequences[:2])

[[1, 289, 733, 1846, 2, 1, 226, 23, 89, 30, 1846, 5, 26, 1, 165, 2, 2833, 2832, 35, 44, 3275, 3, 28, 570, 44, 4823, 6581, 21, 37, 1099, 72, 236, 72, 248, 13, 6580, 72, 158, 1, 376, 2, 1, 289, 733, 1312, 1392, 28, 30, 1846, 13, 3276], [289, 733, 1846, 2, 1, 226, 23, 89, 30, 1846, 5, 26, 1, 165, 2, 2833, 2832, 35, 44, 3275, 3, 28, 570, 44, 4823, 6581, 21, 37, 1099, 72, 236, 72, 248, 13, 6580, 72, 158, 1, 376, 2, 1, 289, 733, 1312, 1392, 28, 30, 1846, 13, 3276, 35]]


In [10]:
# vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

10650


# Tạo tập dữ liệu, tập kết quả

In [11]:
# separate into input and output
sequences = np.array(sequences)
X, y = sequences[:,:-1], sequences[:,-1]
y = tf.keras.utils.to_categorical(y, num_classes=vocab_size)
seq_length = X.shape[1]

In [12]:
print(X.shape)

(219583, 50)


In [13]:
# X_train = X.reshape(X.shape[0], 1, X.shape[1]) # X.reshape(samples, timesteps, features)
X_train = np.asarray(X).astype(np.float64)


In [14]:
X_train  = np.expand_dims(X_train , -1)
print(X_train.shape)

(219583, 50, 1)


In [15]:
print(y.shape)

(219583, 10650)


In [16]:
# y_train = y.reshape(y.shape[0], 1, y.shape[1]) # X.reshape(samples, timesteps, features)
y_train = np.asarray(y)


In [17]:
y_train = np.expand_dims(y_train, -1)
print(y_train.shape)

(219583, 10650, 1)


In [ ]:
y_train = y_train.astype(np.float64)

# Load lại word2vec model

In [ ]:
# # load the whole embedding into memory
# embeddings_index = dict()
# f = open('model.txt')
# for line in f:
# 	values = line.split()
# 	word = values[0]
# 	coefs = np.asarray(values[1:], dtype='float32')
# 	embeddings_index[word] = coefs
# f.close()
# print('Loaded %s word vectors.' % len(embeddings_index))
# print(embeddings_index['i'])

# Tạo ma trận embedding cho từng từ

In [ ]:
# # create a weight matrix for words in training docs
# embedding_matrix = np.zeros((vocab_size, 300))
# for word, i in tokenizer.word_index.items():
# 	embedding_vector = embeddings_index.get(word)
# 	if embedding_vector is not None:
# 		embedding_matrix[i] = embedding_vector
# print(embedding_matrix[1])

# Khởi tạo và cấu hình cho mô hình

In [ ]:
def ELMoEmbedding(x):
    return embed(tf.squeeze(tf.cast(x, tf.string)), signature="default", as_dict=True)["default"]

In [ ]:
# # checkpoint
cp_callback = [
               ModelCheckpoint(
    filepath=checkpoint_path.format('language_model_cbow_weight_en.ckpt'), 
    verbose=1, 
    save_weights_only=True,
    # save_freq='epoch'
    )
               ]

In [ ]:
# define model
model = Sequential()
input_text = Input(shape=(1,), dtype=tf.string)
embedding = Lambda(ELMoEmbedding, 
                  #  output_shape=(1024, )
                   )(input_text)
model.add(embedding)
# model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=seq_length, trainable=False))
model.add(LSTM(100, 
              #  input_shape=(1, 50), 
               return_sequences=True
               ))
model.add(LSTM(100, 
              #  input_shape=(1, 50)
               ))
model.add(Dense(100, activation='relu'))
model.add(vocab_size, activation='softmax'))



# input_text = Input(shape=(50,1), dtype=tf.string)
# embedding = Lambda(ELMoEmbedding, 
#                    output_shape=(1024, )
#                    )(input_text)
# lstm_1 = LSTM(100, 
#               #  input_shape=(1, 50), 
#                return_sequences=True
#                )
# lstm_2 = LSTM(100)
# dense = Dense(100, activation='relu')(embedding)
# pred = Dense(vocab_size, activation='softmax')(dense)
# model = Model(inputs=[input_text], outputs=pred)
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

Huấn luyện mô hình

In [ ]:
# compile model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
# fit model
model.fit(X_train, y_train, batch_size=128, callbacks=cp_callback, epochs=20)
# model.fit(X, y, batch_size=128, callbacks=cp_callback, epochs=20)

In [ ]:
# save the model to file
# model.save(checkpoint_path.format('Language_model_cbow_vnese_final.h5'))
model.save(checkpoint_path.format('Language_model_cbow_eng.h5'))

In [ ]:
# save the tokenizer
dump(tokenizer, open(checkpoint_path.format('tokenizer_cbow_eng.pkl'), 'wb'))
# dump(tokenizer, open(checkpoint_path.format('tokenizer_cbow_vnese.pkl'), 'wb'))